In [1]:
from IPython.core.display import display, HTML
display(HTML(
    '<style>'
        '#notebook { padding-top:0px !important; } ' 
        '.container { width:100% !important; } '
        '.end_space { min-height:0px !important; } '
    '</style>'
))

In [3]:
import findspark
findspark.init()

In [4]:
findspark.find()

'D:\\Softwares\\Spark\\spark-2.4.7-bin-hadoop2.7'

In [5]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setAppName('conf_LR').setMaster('local')
sc = SparkContext(conf=conf)

In [6]:
from pyspark.sql import SparkSession

spark = SparkSession(sc).builder.appName('app_LR').getOrCreate()

In [7]:
df = spark.read.csv("./Resources/Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Linear_Regression/cruise_ship_info.csv", inferSchema=True, header=True)

In [10]:
for i in df.head(5):
    print(i)
    print("\n")

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)


Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)


Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7)


Row(Ship_name='Conquest', Cruise_line='Carnival', Age=11, Tonnage=110.0, passengers=29.74, length=9.53, cabins=14.88, passenger_density=36.99, crew=19.1)


Row(Ship_name='Destiny', Cruise_line='Carnival', Age=17, Tonnage=101.353, passengers=26.42, length=8.92, cabins=13.21, passenger_density=38.36, crew=10.0)




In [11]:
from pyspark.sql.functions import corr

In [12]:
df.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [17]:
df.select(corr('Age','crew'),corr('Tonnage','crew'),corr('passengers','crew'),corr('length','crew'),corr('cabins','crew'),corr('passenger_density','crew')).show()

+-------------------+-------------------+----------------------+------------------+------------------+-----------------------------+
|    corr(Age, crew)|corr(Tonnage, crew)|corr(passengers, crew)|corr(length, crew)|corr(cabins, crew)|corr(passenger_density, crew)|
+-------------------+-------------------+----------------------+------------------+------------------+-----------------------------+
|-0.5306565039638852| 0.9275688115449388|    0.9152341306065384| 0.895856627101658|0.9508226063578497|         -0.15550928421699717|
+-------------------+-------------------+----------------------+------------------+------------------+-----------------------------+



In [18]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler, StringIndexer

In [19]:
indexer = StringIndexer(inputCol='Cruise_line',outputCol='Cruise_cat')

In [20]:
indexed_data = indexer.fit(df).transform(df)

In [26]:
indexed_data.groupBy(['Cruise_line','Cruise_cat']).count().orderBy('Cruise_cat').show()

+-----------------+----------+-----+
|      Cruise_line|Cruise_cat|count|
+-----------------+----------+-----+
|  Royal_Caribbean|       0.0|   23|
|         Carnival|       1.0|   22|
|         Princess|       2.0|   17|
| Holland_American|       3.0|   14|
|        Norwegian|       4.0|   13|
|            Costa|       5.0|   11|
|        Celebrity|       6.0|   10|
|              MSC|       7.0|    8|
|             Star|       8.0|    6|
|              P&O|       9.0|    6|
|Regent_Seven_Seas|      10.0|    5|
|        Silversea|      11.0|    4|
|          Oceania|      12.0|    3|
|         Seabourn|      13.0|    3|
|         Windstar|      14.0|    3|
|           Cunard|      15.0|    3|
|          Azamara|      16.0|    2|
|           Disney|      17.0|    2|
|          Crystal|      18.0|    2|
|           Orient|      19.0|    1|
+-----------------+----------+-----+



In [27]:
indexed_data.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_cat']

In [28]:
assembler = VectorAssembler(inputCols=['Age', 'Tonnage', 'passengers', 'length', 'cabins', 'passenger_density', 'Cruise_cat'], outputCol='features')

In [32]:
opData = assembler.transform(indexed_data)

In [33]:
opData.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------+--------------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_cat|            features|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------+--------------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|      16.0|[6.0,30.276999999...|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|      16.0|[6.0,30.276999999...|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|       1.0|[26.0,47.262,14.8...|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|       1.0|[11.0,110.0,29.74...|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0| 

In [34]:
final_data = opData.select(['features','crew'])

In [35]:
final_data.show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [37]:
lr = LinearRegression(labelCol='crew')

In [47]:
train, test = final_data.randomSplit([0.7,0.3])

In [48]:
lr_model = lr.fit(train)

In [49]:
test_res = lr_model.evaluate(test)

In [50]:
test_res.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|   0.404360282818935|
| -1.4458116148422828|
| -0.9201451558229659|
|   0.527277983236262|
|-0.14576878941587879|
| -1.3711306653853228|
| 0.33576274552902596|
| -0.8538055667807996|
| -0.6498801646613099|
|  1.8790492006247592|
|0.040435029874361206|
|  -0.634665824022866|
|  -0.539856758178054|
| -1.0687363374746557|
|-0.00772009693177...|
|  -0.273788626373177|
|   7.049161386647624|
|  0.3238072413253388|
| -0.5979576073982589|
|  0.7172087196585046|
+--------------------+
only showing top 20 rows



In [51]:
test_res.r2

0.89434229582991

In [52]:
test_res.rootMeanSquaredError

1.2111490259420394

In [53]:
final_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              158|
|   mean|7.794177215189873|
| stddev|3.503486564627034|
|    min|             0.59|
|    max|             21.0|
+-------+-----------------+

